# 线性模型及其实现

## 线性回归

这部分我们使用波士顿房价数据集来说明。这个数据集包含波士顿各个区域的多个特征，比如犯罪率、一氧化氮浓度、户主的年龄信息等，综合这些信息来估计某处房子的售价。

In [1]:
## 导入所需的数据集和包
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LinearRegression

In [2]:
## 加载数据集
boston = datasets.load_boston()

In [3]:
## 查看数据集相关信息
print boston.DESCR

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [4]:
X = boston.data
y = boston.target

In [5]:
lm = LinearRegression()

In [6]:
## 选取前400个数据作为训练集，剩下的作为测试集
X_train = X[:400, :]
y_train = y[:400]
X_test = X[400:,:]
y_test = y[400:]

In [7]:
print "训练集的规模为 %s，测试集的规模为 %s" % (str(X_train.shape), str(X_test.shape))

训练集的规模为 (400L, 13L)，测试集的规模为 (106L, 13L)


In [8]:
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
## 输出所有的参数
print lm.intercept_, lm.coef_

28.672599590855004 [-1.91246374e-01  4.42289967e-02  5.52207977e-02  1.71631351e+00
 -1.49957220e+01  4.88773025e+00  2.60921031e-03 -1.29480799e+00
  4.84787214e-01 -1.54006673e-02 -8.08795026e-01 -1.29230427e-03
 -5.17953791e-01]


然后我们就可以用它来做预测了。

In [10]:
y_pred = lm.predict(X_test)

因为是回归问题，所以我们用 mean square error (mse) 来检验拟合程度的好坏。具体来说就是用这个模型去算一下测试集的 X ，看看结果跟测试集的 y 的 mse 有多大。

In [11]:
print 'mse = %.4f' % np.mean(np.square(y_pred - y_test))

mse = 38.1643


## 特征选取 Feature selection

但是 LinearRegression 这个函数并不提供可供选择特征的参数。因为有些特征跟我们最终的 y 关系不大，如果硬要把它加在线性模型里，反而会造成更大的误差。

所以这里我们另外导入一个包来做这件事：

In [12]:
import statsmodels.api as sm

C:\Users\Hikaru\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


这个包的设定跟 sklearn 略有不同。它预设的模型就是 $y = \boldsymbol{w}^T\boldsymbol{x}$ ，没有 intercept 的部分。所以我们这里要手动加上一个常数项。

In [13]:
X_train_1 = sm.add_constant(X_train)

接着用 sm.OLS 来做线性回归。OLS=Ordinary Least Square 普通最小二乘。注意这里调用函数的时候要把y放前面。

In [14]:
lm_sm = sm.OLS(y_train, X_train_1)
results = lm_sm.fit()

In [15]:
print results.params

[ 2.86725996e+01 -1.91246374e-01  4.42289967e-02  5.52207977e-02
  1.71631351e+00 -1.49957220e+01  4.88773025e+00  2.60921031e-03
 -1.29480799e+00  4.84787214e-01 -1.54006673e-02 -8.08795026e-01
 -1.29230427e-03 -5.17953791e-01]


可以看到这里的 params 跟上面 (intercept, coef) 的数值是一样的。

那么问题来了，要如何做简单的特征选取呢？

In [16]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.725
Method:                 Least Squares   F-statistic:                     81.87
Date:                Fri, 04 May 2018   Prob (F-statistic):          2.92e-102
Time:                        15:28:21   Log-Likelihood:                -1188.5
No. Observations:                 400   AIC:                             2405.
Df Residuals:                     386   BIC:                             2461.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.6726      6.152      4.661      0.000      16.578      40.768
x1            -0.1912      0.054     -3.539      0.000      -0.297      -0.085
x2             0.0442      0.014      3.134      0.002       0.016       0.072
x3             0.0552      0.066      0.843      0.400      -0.074       0.184
x4             1.7163      0.891      1.926      0.055      -0.036       3.468
x5           -14.9957      4.558     -3.290      0.001     -23.957      -6.035
x6             4.8877      0.485     10.079      0.000       3.934       5.841
x7             0.0026      0.014      0.182      0.856      -0.026       0.031
x8            -1.2948      0.212     -6.116      0.000      -1.711      -0.879
x9             0.4848      0.087      5.550      0.000       0.313       0.657
x10           -0.0154      0.004     -3.463      0.001      -0.024      -0.007
x11           -0.8088      0.140     -5.774      0.000      -1.084      -0.533
x12           -0.0013      0.007     -0.198      0.843      -0.014       0.012
x13           -0.5180      0.060     -8.704      0.000      -0.635      -0.401
==============================================================================
Omnibus:                      116.345   Durbin-Watson:                   1.124
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              538.938
Skew:                           1.177   Prob(JB):                    9.36e-118
Kurtosis:                       8.176   Cond. No.                     1.52e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.52e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

这个 summary 结果我们首先关心的是右上角的 R-squared （决定系数）。一般来说这个值是正的。假如它的值变成了负数，说明当前模型非常不适合这个数据集，需要更换别的模型。

第二张表的 coef 列就是前面的 params 。同一张表我们还需要关心的是 P>|t| 那一列。假如这里的数值大于0.05就可以考虑把相应的特征踢掉了。在里就是x3, x7, x12，对应变量名为:

In [17]:
print boston.feature_names[np.array([2,6,11])]

['INDUS' 'AGE' 'B']


INDUS 和 AGE 都比较好理解，B 出现在这里以我们对美国的了解肯定是不科学的。这里很有可能跟数据集的样本分布有关。因为我们划分训练集的时候是粗暴地取了前400个，而不是随机地抽取400个样本。

请根据这个结论重新划分训练集/数据集进行分析，再根据结果做适当的特征筛选，观察筛选的结果是否降低了错误率。

## 逻辑回归

这部分我们用股票数据集 Smarket 来说明，数据放在Smarket.csv中。把它放在跟这个notebook同一个文件夹下即可。

In [18]:
from sklearn.linear_model import LogisticRegression
import pandas as pd

这里我们导入了另一个包 pandas，这也是一个 Python 数据分析里常用的包。用它可以一次性读入一个 csv 文件。它的功能非常强大。后面要用到的话再慢慢说。

In [19]:
smarket = pd.read_csv('Smarket.csv')

In [20]:
## 用shape函数查看数据集规模
smarket.shape

(1250, 9)

In [21]:
## 查看各个 column (feature) 的名称
smarket.columns

Index([u'Year', u'Lag1', u'Lag2', u'Lag3', u'Lag4', u'Lag5', u'Volume',
       u'Today', u'Direction'],
      dtype='object')

这个数据集包含2001年到2005年间1250天 S&P 500 股票指数的投资回报数据。Year 表示年份，Lag1~Lag5表示过去1到5个交易日的投资回报率。Today是今日的投资回报率。Direction是市场走势，它有两类，Up和Down。另外还有一个特征是Volumn，表示前一日的成交量，单位是billion。

In [22]:
## 查看数据的整体情况
smarket.describe()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
count,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.00000,1250.000000,1250.000000
mean,2003.016000,0.003834,0.003919,0.001716,0.001636,0.00561,1.478305,0.003138
std,1.409018,1.136299,1.136280,1.138703,1.138774,1.14755,0.360357,1.136334
min,2001.000000,-4.922000,-4.922000,-4.922000,-4.922000,-4.92200,0.356070,-4.922000
25%,2002.000000,-0.639500,-0.639500,-0.640000,-0.640000,-0.64000,1.257400,-0.639500
50%,2003.000000,0.039000,0.039000,0.038500,0.038500,0.03850,1.422950,0.038500
75%,2004.000000,0.596750,0.596750,0.596750,0.596750,0.59700,1.641675,0.596750
max,2005.000000,5.733000,5.733000,5.733000,5.733000,5.73300,3.152470,5.733000


In [23]:
## 查看前面几行
smarket.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In [24]:
## 数据类型换回 ndarray
## 这里我们不考虑 Year 这个特征，因为它的数量级跟其他特征差太多了
X = np.array(smarket.iloc[:,1:7])
y = np.array(smarket['Direction'])

In [25]:
X.shape

(1250L, 6L)

同样的， sklearn 提供了直接调用 Logistic Regression 的函数 LogisticRegression()，使用方法同其他的机器学习函数一样。

In [26]:
logit = LogisticRegression()
logit.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
y_pred = logit.predict(X)

In [28]:
## calculate training error
print 'training error= %.4f' % (1 - np.mean(y_pred == y))

training error= 0.4752


In [29]:
print y_pred[:10]

['Up' 'Down' 'Down' 'Up' 'Up' 'Up' 'Down' 'Up' 'Up' 'Down']


可以看看 y_pred 的结果同样是两类，那我们要怎么看每个样本预测的概率呢？可以使用 predict_proba 函数

In [30]:
y_predProbs = logit.predict_proba(X)

In [31]:
y_predProbs[:10]

array([[0.49224404, 0.50775596],
       [0.51791235, 0.48208765],
       [0.51852092, 0.48147908],
       [0.48438969, 0.51561031],
       [0.48865013, 0.51134987],
       [0.49273069, 0.50726931],
       [0.50711612, 0.49288388],
       [0.49058882, 0.50941118],
       [0.48177107, 0.51822893],
       [0.51055965, 0.48944035]])

看前10个样本的预测结果，左边一列代表分类结果为 Down 的概率，右边一列代表分类结果为 Up 的概率。
不过看这个举棋不定的样子，大概就跟瞎猜差不多。

炒股有风险，炒股需谨慎。
拿来练练手还是可以的。

我们也可以用 statsmodel 中的函数来做逻辑回归的同时筛选特征，做法有点类似于前面的线性回归。但是我们这里用更 formula 的方式来表示，这样看起来更科学一些。
首先导入statsmodels.formula.api

In [32]:
import statsmodels.formula.api as smf

然后用glm函数来计算。glm=Generalize Linear Model 广义线性模型。当family=Binomial的时候就是用逻辑回归做二元分类。在 formula 部分我们指定需要预测的 column name 和 成为特征的 column name, 表现为线性求和的方式。对应我们的线性模型。在 data 的部分制定数据集就可以了。是不是很方便！

In [33]:
logit_sm = smf.glm(formula='Direction~Lag1+Lag2+Lag3+Lag4+Lag5+Volume', data=smarket,
                  family=sm.families.Binomial())

In [34]:
results = logit_sm.fit()

In [35]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          Generalized Linear Model Regression Results                           
================================================================================================
Dep. Variable:     ['Direction[Down]', 'Direction[Up]']   No. Observations:                 1250
Model:                                              GLM   Df Residuals:                     1243
Model Family:                                  Binomial   Df Model:                            6
Link Function:                                    logit   Scale:                             1.0
Method:                                            IRLS   Log-Likelihood:                -863.79
Date:                                  Fri, 04 May 2018   Deviance:                       1727.6
Time:                                          15:30:22   Pearson chi2:                 1.25e+03
No. Iterations:                                       4                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1260      0.241      0.523      0.601      -0.346       0.598
Lag1           0.0731      0.050      1.457      0.145      -0.025       0.171
Lag2           0.0423      0.050      0.845      0.398      -0.056       0.140
Lag3          -0.0111      0.050     -0.222      0.824      -0.109       0.087
Lag4          -0.0094      0.050     -0.187      0.851      -0.107       0.089
Lag5          -0.0103      0.050     -0.208      0.835      -0.107       0.087
Volume        -0.1354      0.158     -0.855      0.392      -0.446       0.175
==============================================================================
"""

In [36]:
y_pred = results.predict()

In [37]:
y_pred_class = np.array(['Down']*len(y))
y_pred_class[y_pred < 0.5] = 'Up'

In [38]:
print 'training error=%.4f' % (1 - np.mean(y_pred_class == y))

training error=0.4784


这个错误率，也跟瞎猜差不多。但是我们注意到在Lag1和Lag2这两个特征的P值相对于其它特征来说还是比较小的，假如我们把别的都踢掉会怎么样呢？

In [39]:
logit_sm_less = smf.glm(formula='Direction~Lag1+Lag2', data=smarket,
                  family=sm.families.Binomial())

In [40]:
results_less = logit_sm_less.fit()

In [41]:
results_less.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          Generalized Linear Model Regression Results                           
================================================================================================
Dep. Variable:     ['Direction[Down]', 'Direction[Up]']   No. Observations:                 1250
Model:                                              GLM   Df Residuals:                     1247
Model Family:                                  Binomial   Df Model:                            2
Link Function:                                    logit   Scale:                             1.0
Method:                                            IRLS   Log-Likelihood:                -864.20
Date:                                  Fri, 04 May 2018   Deviance:                       1728.4
Time:                                          15:30:37   Pearson chi2:                 1.25e+03
No. Iterations:                                       4                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0742      0.057     -1.310      0.190      -0.185       0.037
Lag1           0.0715      0.050      1.427      0.153      -0.027       0.170
Lag2           0.0445      0.050      0.890      0.374      -0.054       0.142
==============================================================================
"""

In [42]:
y_pred = results_less.predict()
y_pred_class = np.array(['Down']*len(y))
y_pred_class[y_pred < 0.5] = 'Up'
print 'training error=%.4f' % (1 - np.mean(y_pred_class == y))

training error=0.4720


这个结果看起来也就好了一点点点吧……

练习：以2005年之前的数据为训练集，2005年的数据为测试集，计算logistic regression在测试集上的testing error